#Machine Learning Models for prepared significant variables dataset

**In this notebook I'm utilizing five Machine Learning algorithms and one Deep Learning algorithm on initially cleaned dataset. Making use of RandomizedSearch in pipelines to find out best hyperparameters for ML algorithms. I'll perform some additional preparations of dataset, divide into train and test subsets, encoding into numbers with pandas get_dummies and OrdinalEncoder, using StandardScaller for scaling, SMOTEENN to make classes equal and PCA to decrease amount of variables**

Imports:

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import (StandardScaler, 
                                   OrdinalEncoder, 
                                   MinMaxScaler)

from sklearn.model_selection import (train_test_split, 
                                     GridSearchCV, 
                                     StratifiedKFold, 
                                     RandomizedSearchCV)

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import (classification_report, 
                             roc_auc_score, 
                             make_scorer, 
                             recall_score, 
                             confusion_matrix, 
                             accuracy_score,
                            get_scorer_names)
from sklearn.decomposition import PCA

Loading dataset:

In [2]:
data_clean = pd.read_pickle("data/data_clear.pkl")

Dividing into predictor variables X and target y ("is_canceled"):

In [3]:
X = data_clean.drop("is_canceled", axis=1)
y = data_clean.is_canceled

Splitting dataset into train and test subsets with test size 30% and train 70%:

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    stratify=y,
                                                    random_state=42
                                                   )

Shape after division

In [5]:
X_train.shape

(83573, 27)

In [6]:
X_test.shape

(35817, 27)

Inputting NaNs in country column with the most frequent value ()max of train subset into train and test:

In [7]:
country_input = X_train["country"][X_train.country.value_counts().max()]

In [8]:
X_train.country.fillna(country_input, inplace=True)

In [9]:
X_test.country.fillna(country_input, inplace=True)

Inputting NaNs in agent column with the most frequent value ()max of train subset into train and test:

In [10]:
agent_input = X_train["country"][X_train.agent.value_counts().max()]

In [11]:
X_train.agent.fillna(agent_input, inplace=True)

In [12]:
X_test.agent.fillna(agent_input, inplace=True)

Outlier value of column adr found in a file "Data_Preparations" now is to be replaced with mean of adr column.

In [ ]:
(X_train["adr"]==5400).sum()

In [ ]:
(X_test["adr"]==5400).sum()

In [ ]:
if (X_train["adr"]==5400).sum() > 0:
    X_train.replace({5400.0:np.round(X_train.adr.mean(), 2)}, inplace=True) #filling inordinary adr value with mean of training set adr column
    print("Outlier observations in train subset = ", (X_train["adr"]==5400).sum())
elif (X_test["adr"]==5400).sum() > 0:
    X_test.replace({5400.0:np.round(X_train.adr.mean(), 2)}, inplace=True)
    print("Outlier observations in test subset = ", (X_test["adr"]==5400).sum())

Encoding columns of most numerous classes with OrdinalEncoder:

In [ ]:
data_label_train = X_train[["agent", "company", "country", "reservation_status_date", "arrival_date"]]
data_label_test = X_test[["agent", "company", "country", "reservation_status_date", "arrival_date"]]

In [ ]:
ode = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
ode.fit(data_label_train)
data_label_train_ode = pd.DataFrame(ode.transform(data_label_train),
                                    columns=["agent", "company", "country", "reservation_status_date", "arrival_date"])
data_label_test_ode = pd.DataFrame(ode.transform(data_label_test), 
                                   columns=["agent", "company", "country", "reservation_status_date", "arrival_date"])

In [ ]:
data_label_train_ode

Updating encoded columns:

In [ ]:
X_train.drop(["agent", "company", "country", "reservation_status_date", "arrival_date"], axis=1, inplace=True)
X_test.drop(["agent", "company", "country", "reservation_status_date", "arrival_date"], axis=1, inplace=True)

In [ ]:
X_train = pd.concat([X_train.reset_index(drop=True), data_label_train_ode.reset_index(drop=True)], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), data_label_test_ode.reset_index(drop=True)], axis=1)

In [ ]:
X_train.shape

Encoding training and test subsets with get_dummies:

In [ ]:
X_train = pd.get_dummies(X_train, drop_first=True)

In [ ]:
X_test = pd.get_dummies(X_test, drop_first=True)
X_test = X_test.reindex(columns = X_train.columns, fill_value=0)

In [ ]:
X_train.shape

Initiating StandardScaler for further data scaling:

In [ ]:
scaler = StandardScaler()

Initiating Principal Components with ten components reducing dimentions to ten components :

In [ ]:
pca = PCA(n_components=10)

Initiating algorithm to ballance unballanced data- SMOTEENN:

In [ ]:
SMOTEEN = SMOTEENN()

RandomForestClassifier algorythm with RandomizedGridSearch in pipeline, scaling reducing, ballancing:

In [ ]:
stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=11)
#imbpipeline
pipeline_rf = imbpipeline(steps=[
    ['scaler', scaler],
    ['pca', pca],
    ['smote', SMOTEEN],
    ['rf', RandomForestClassifier()]])
    
param_distributions_rf = {
    'rf__n_estimators': [20, 100, 300],
    'rf__max_depth': [10, 20],
    'rf__min_samples_split': [5, 10],
    'pca__n_components': [5, 10, 20]
}

search_rf = RandomizedSearchCV(pipeline_rf, 
                               param_distributions_rf, 
                               n_iter=10, 
                               cv=stratified_kfold, 
                               scoring='roc_auc',
                               verbose=3
                              )

search_rf.fit(X_train, y_train)
y_pred_rf = search_rf.best_estimator_.predict(X_test)
print("Random Forest:")
print(search_rf.best_params_)
print(f'Results on test: {search_rf.best_estimator_.score(X_test, y_test)}')
print(f'Results on train: {search_rf.best_estimator_.score(X_train, y_train)}')

Achieving scores of classification, saving accuracy, recall and F1 score in data frame:

Best hyperparameters:

In [ ]:
search_rf.best_params_

In [ ]:
#print(get_scorer_names())

In [ ]:
y_pred_rf

In [ ]:
print(classification_report(y_test, y_pred_rf))

In [ ]:
A_report_rf = pd.DataFrame(classification_report(y_test, y_pred_rf, output_dict=True))

In [ ]:
for i, name in enumerate(A_report_rf.columns):
  A_report_rf = A_report_rf.rename(columns={(A_report_rf.iloc[:,i].name): ('RF_'+A_report_xgb.iloc[:,i].name)})

In [ ]:
A_report_rf

DecisionTreeClassifier algorythm with RandomizedGridSearch in pipeline, scaling reducing, ballancing:

In [ ]:
stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=13)

pipeline = imbpipeline(steps = [['scaler', scaler],
                                ['pca', pca],
                                ['smote', SMOTEEN],
                                ['dtc', DecisionTreeClassifier()]])

    
param_grid = {'dtc__max_leaf_nodes' : [2, 5, 10, 30], 
             'dtc__max_depth': [4, 10, 20, 40],
             'dtc__random_state' : [23],
             'pca__n_components': [5, 10, 20]
             }

search_dtc = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='roc_auc',
                           cv=stratified_kfold,                           
                          verbose=3,
                           #n_jobs=3
                         )

search_dtc.fit(X_train, y_train)
y_pred_dtc = search_dtc.best_estimator_.predict(X_test)
cv_score = search_dtc.best_score_
test_score = search_dtc.score(X_test, y_test)
print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')
print("Decision Tree:")
print(search_rf.best_params_)
print(f'Results on test: {search_rf.best_estimator_.score(X_test, y_test)}')
print(f'Results on train: {search_rf.best_estimator_.score(X_train, y_train)}')

Achieving scores of classification, saving accuracy, recall and F1 score in data frame:

Best hyperparameters:

In [ ]:
search_dtc.best_params_

In [ ]:
y_pred_dtc

In [ ]:
print(classification_report(y_test, y_pred_dtc))
A_report_dtc = pd.DataFrame(classification_report(y_test, y_pred_dtc, output_dict=True))

In [ ]:
for i, name in enumerate(A_report_dtc.columns):
  A_report_dtc = A_report_dtc.rename(columns={(A_report_dtc.iloc[:,i].name): ('DTC_'+A_report_dtc.iloc[:,i].name)})


In [ ]:
A_report_dtc

Support Vector Classifier algorythm with RandomizedGridSearch in pipeline, scaling reducing, ballancing:

In [ ]:
stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=23)

pipeline_SVC = imbpipeline([('scaler', scaler),
                            ('pca', pca),
                            ('SMOTE', SMOTEEN),
                            ('SVC', SVC())])
    
params_SVC = {
              'SVC__gamma': ['auto'],
              'SVC__max_iter': [150, 300, 500],
              'SVC__decision_function_shape': ['ovo'],
              'SVC__degree': [1],
              'SVC__kernel': ['rbf'],
              'SVC__random_state': [11],
              'pca__n_components': [5, 10, 20]
             }

search_SVC = GridSearchCV(pipeline_SVC,
                             params_SVC,
                             scoring='roc_auc',
                             cv=stratified_kfold,
                            verbose=3,
                            #n_jobs=3
                         )

search_SVC.fit(X_train, y_train)

cv_score = search_SVC.best_score_
test_score = search_SVC.score(X_test, y_test)
print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')
print("Support Vector:")
print(search_SVC.best_params_)
print(f'Results on test: {search_SVC.best_estimator_.score(X_test, y_test)}')
print(f'Results on train: {search_SVC.best_estimator_.score(X_train, y_train)}')


Achieving scores of classification, saving accuracy, recall and F1 score in data frame:

In [ ]:
y_pred_SVC_train = search_SVC.best_estimator_.predict(X_train)

In [ ]:
y_pred_svc_test = search_SVC.best_estimator_.predict(X_test)

In [ ]:
y_pred_SVC = search_SVC.predict(X_test)

Best hyperparameters:

In [ ]:
search_SVC.best_params_

In [ ]:
print(classification_report(y_test, y_pred_SVC))
A_report_svc = pd.DataFrame(classification_report(y_test, y_pred_SVC, output_dict=True))

In [ ]:
A_report_svc

In [ ]:
for i, name in enumerate(A_report_svc.columns):
  A_report_svc = A_report_svc.rename(columns={(A_report_svc.iloc[:,i].name): ('SVC_'+A_report_svc.iloc[:,i].name)})


In [ ]:
A_report_svc

XGBClassifier algorythm with RandomizedGridSearch in pipeline, scaling reducing, ballancing:

In [ ]:
stratified_kfold = StratifiedKFold(n_splits=3,
                                       shuffle=True,
                                       random_state=77)

pipeline = imbpipeline(steps=[('scaler', scaler),
                              ('pca', pca),
                              ('smote', SMOTEEN),
                              ('XGB', XGBClassifier())])

params = {
    'XGB__n_estimators': [100, 500, 800],
    'XGB__max_depth': [3, 5, 10],
    'XGB__learning_rate': [0.1, 0.5],
    'pca__n_components': [5, 10, 20]
    }

search_XGB = GridSearchCV(pipeline, 
                          params, 
                          scoring='roc_auc', 
                          cv=stratified_kfold, 
                          verbose=3,
                        #n_jobs=3
                         ) 

search_XGB.fit(X_train, y_train) 
accuracy_score(y_test, search_XGB.predict(X_test))

Achieving scores of classification, saving accuracy, recall and F1 score in data frame:

Best hyperparameters:

In [ ]:
search_XGB.best_params_

In [ ]:
#XGBClassifier().get_params().keys()

In [ ]:
search_XGB.cv_results_["mean_test_score"]

In [ ]:
accuracy_score(y_test, search_XGB.predict(X_test))

In [ ]:
y_pred_XGB = search_XGB.best_estimator_.predict(X_test)
test_score = search_XGB.score(X_test, y_test)
cv_score = search_XGB.best_score_

In [ ]:
print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')
print("XGBClassifier:")
print(search_XGB.best_params_)
print(f'Results on test: {search_XGB.best_estimator_.score(X_test, y_test)}')
print(f'Results on train: {search_XGB.best_estimator_.score(X_train, y_train)}')

In [ ]:
print(classification_report(y_test, y_pred_XGB))
A_report_xgb = pd.DataFrame(classification_report(y_test, y_pred_XGB, output_dict=True))

In [ ]:
for i, name in enumerate(A_report_xgb.columns):
  A_report_xgb = A_report_xgb.rename(columns={(A_report_xgb.iloc[:,i].name): ('XGB_'+A_report_xgb.iloc[:,i].name)})


In [ ]:
A_report_xgb

LogisticRegression algorythm with RandomizedGridSearch in pipeline, scaling reducing, ballancing:

In [ ]:
pipeline = imbpipeline(steps = [['scaler', scaler],
                                ['pca', pca],
                                ['smote', SMOTEEN],
                                ['LR', LogisticRegression()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=13)
    
param_grid = {'LR__C':[20, 50, 70],
             'LR__random_state': [11],
             'LR__multi_class': ['auto'],
             'LR__max_iter': [100, 200, 500],
             'LR__solver': ['saga'],
             'LR__penalty': ['l2', 'l1'],
             'pca__n_components': [5, 10, 20]
             }
                                                                 
search_LR = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='roc_auc',
                           cv=stratified_kfold,
                           verbose=3,
                           #n_jobs=3
                        )

search_LR.fit(X_train, y_train)
cv_score = search_LR.best_score_
test_score = search_LR.score(X_test, y_test)
print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')
print("XGBClassifier:")
print(search_LR.best_params_)
print(f'Results on test: {search_LR.best_estimator_.score(X_test, y_test)}')
print(f'Results on train: {search_LR.best_estimator_.score(X_train, y_train)}')

Achieving scores of classification, saving accuracy, recall and F1 score in data frame:

Best hyperparameters:

In [ ]:
search_LR.best_params_

In [ ]:
y_pred_lr = search_LR.best_estimator_.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_lr))
A_report_lr = pd.DataFrame(classification_report(y_test, y_pred_lr, output_dict=True))

In [ ]:
for i, name in enumerate(A_report_lr.columns):
  A_report_lr = A_report_lr.rename(columns={(A_report_lr.iloc[:,i].name): ('LR_'+A_report_lr.iloc[:,i].name)})


In [ ]:
A_report_lr

Utilizing Multi Layer Perceptron algorythm with RandomizedGridSearch in pipeline, scaling reducing, ballancing:

In [ ]:
pipeline = imbpipeline(steps = [['scaler', scaler],
                                ['pca', pca],
                                ['smote', SMOTEEN],
                                ['MLP', MLPClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=13)
    
param_grid = {'MLP__hidden_layer_sizes':[8, 4, 16],
             'MLP__activation': ['relu'],
              'MLP__solver': ['adam'],
              'MLP__random_state': [42],
              'MLP__max_iter': [1000],
              'MLP__batch_size': [32],
              'pca__n_components': [5, 10, 20]
             }
                                                                 
search_MLP = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='roc_auc',
                           cv=stratified_kfold,
                           verbose=3,
                           #n_jobs=3
                        )

search_MLP.fit(X_train, y_train)
cv_score = search_MLP.best_score_
test_score = search_MLP.score(X_test, y_test)
print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')

Achieving scores of classification, saving accuracy, recall and F1 score in data frame:

Best hyperparameters:

In [ ]:
search_MLP.best_params_

In [ ]:
y_pred_mlp = search_MLP.predict(X_test)
print(classification_report(y_test, y_pred_mlp))
A_report_mlp = pd.DataFrame(classification_report(y_test, y_pred_mlp, output_dict=True))

In [ ]:
for i, name in enumerate(A_report_mlp.columns):
  A_report_mlp = A_report_mlp.rename(columns={(A_report_mlp.iloc[:,i].name): ('MLP_'+A_report_mlp.iloc[:,i].name)})


In [ ]:
A_report_mlp

Creating Data Frame containing all six classifiers results:

In [ ]:
A_results = pd.concat([A_report_rf, 
                       A_report_dtc, 
                       A_report_svc, 
                       A_report_xgb, 
                       A_report_lr, 
                       A_report_mlp], 
                      axis=1)

In [ ]:
A_results

Saving results in a file:

In [ ]:
A_results.to_pickle("data/A_dataset_results.pkl")

In [50]:
A_results = pd.read_pickle("data/A_dataset_results.pkl")

In [15]:
A_results

,RF_0,RF_1,RF_accuracy,RF_macro avg,RF_weighted avg,DTC_0,DTC_1,DTC_accuracy,DTC_macro avg,DTC_weighted avg,...,LR_0,LR_1,LR_accuracy,LR_macro avg,LR_weighted avg,MLP_0,MLP_1,MLP_accuracy,MLP_macro avg,MLP_weighted avg
precision,0.798638,0.791443,0.796577,0.795041,0.795973,0.801758,0.639499,0.738448,0.720629,0.741656,...,0.830833,0.651304,0.755898,0.741069,0.764334,0.826294,0.776390,0.810006,0.801342,0.807809
recall,0.905100,0.612120,0.796577,0.758610,0.796577,0.776585,0.673626,0.738448,0.725106,0.738448,...,0.768825,0.733926,0.755898,0.751376,0.755898,0.884080,0.684103,0.810006,0.784092,0.810006
f1-score,0.848543,0.690326,0.796577,0.769435,0.789938,0.788971,0.656119,0.738448,0.722545,0.739761,...,0.798627,0.690151,0.755898,0.744389,0.758447,0.854211,0.727331,0.810006,0.790771,0.807213
support,22550.000000,13267.000000,0.796577,35817.000000,35817.000000,22550.000000,13267.000000,0.738448,35817.000000,35817.000000,...,22550.000000,13267.000000,0.755898,35817.000000,35817.000000,22550.000000,13267.000000,0.810006,35817.000000,35817.000000
